In [6]:
# import python modules -- this cell needs to be run again if you make changes to any of the files
import matplotlib.pyplot as plt
import numpy as np

from IPython.core.display import HTML
from itertools import chain
from collections import Counter, defaultdict
from helpers import show_model, Dataset
from pomegranate import State, HiddenMarkovModel, DiscreteDistribution

In [7]:
data = Dataset("tags-universal.txt", "brown-universal.txt", train_test_split=0.8)

print("There are {} sentences in the corpus.".format(len(data)))
print("There are {} sentences in the training set.".format(len(data.training_set)))
print("There are {} sentences in the testing set.".format(len(data.testing_set)))

assert len(data) == len(data.training_set) + len(data.testing_set), \
       "The number of sentences in the training set + testing set should sum to the number of sentences in the corpus"

There are 57340 sentences in the corpus.
There are 45872 sentences in the training set.
There are 11468 sentences in the testing set.


In [8]:
key = 'b100-39560'
print("Sentence: {}".format(key))
print("words:\n\t{!s}".format(data.sentences[key].words))
print("tags:\n\t{!s}".format(data.sentences[key].tags))

Sentence: b100-39560
words:
	('We', 'ran', 'east', 'for', 'about', 'half', 'a', 'mile', 'before', 'we', 'turned', 'back', 'to', 'the', 'road', ',', 'panting', 'from', 'the', 'effort', 'and', 'soaked', 'with', 'sweat', '.')
tags:
	('PRON', 'VERB', 'NOUN', 'ADP', 'ADV', 'PRT', 'DET', 'NOUN', 'ADP', 'PRON', 'VERB', 'ADV', 'ADP', 'DET', 'NOUN', '.', 'VERB', 'ADP', 'DET', 'NOUN', 'CONJ', 'VERB', 'ADP', 'NOUN', '.')


In [9]:
print("There are a total of {} samples of {} unique words in the corpus."
      .format(data.N, len(data.vocab)))
print("There are {} samples of {} unique words in the training set."
      .format(data.training_set.N, len(data.training_set.vocab)))
print("There are {} samples of {} unique words in the testing set."
      .format(data.testing_set.N, len(data.testing_set.vocab)))
print("There are {} words in the test set that are missing in the training set."
      .format(len(data.testing_set.vocab - data.training_set.vocab)))

assert data.N == data.training_set.N + data.testing_set.N, \
       "The number of training + test samples should sum to the total number of samples"

There are a total of 1161192 samples of 56057 unique words in the corpus.
There are 928458 samples of 50536 unique words in the training set.
There are 232734 samples of 25112 unique words in the testing set.
There are 5521 words in the test set that are missing in the training set.


In [10]:
# accessing words with Dataset.X and tags with Dataset.Y 
for i in range(2):    
    print("Sentence {}:".format(i + 1), data.X[i])
    print()
    print("Labels {}:".format(i + 1), data.Y[i])
    print()

Sentence 1: ('Mr.', 'Podger', 'had', 'thanked', 'him', 'gravely', ',', 'and', 'now', 'he', 'made', 'use', 'of', 'the', 'advice', '.')

Labels 1: ('NOUN', 'NOUN', 'VERB', 'VERB', 'PRON', 'ADV', '.', 'CONJ', 'ADV', 'PRON', 'VERB', 'NOUN', 'ADP', 'DET', 'NOUN', '.')

Sentence 2: ('But', 'there', 'seemed', 'to', 'be', 'some', 'difference', 'of', 'opinion', 'as', 'to', 'how', 'far', 'the', 'board', 'should', 'go', ',', 'and', 'whose', 'advice', 'it', 'should', 'follow', '.')

Labels 2: ('CONJ', 'PRT', 'VERB', 'PRT', 'VERB', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADP', 'ADP', 'ADV', 'ADV', 'DET', 'NOUN', 'VERB', 'VERB', '.', 'CONJ', 'DET', 'NOUN', 'PRON', 'VERB', 'VERB', '.')



In [11]:
# use Dataset.stream() (word, tag) samples for the entire corpus
print("\nStream (word, tag) pairs:\n")
for i, pair in enumerate(data.stream()):
    print("\t", pair)
    if i > 5: break


Stream (word, tag) pairs:

	 ('Mr.', 'NOUN')
	 ('Podger', 'NOUN')
	 ('had', 'VERB')
	 ('thanked', 'VERB')
	 ('him', 'PRON')
	 ('gravely', 'ADV')
	 (',', '.')


In [12]:
print('there are', len(list(data.training_set.tagset)), 'unique POS which are:')
print(list(data.training_set.tagset))

there are 12 unique POS which are:
['NUM', 'ADV', 'VERB', 'CONJ', 'PRON', '.', 'ADP', 'NOUN', 'DET', 'ADJ', 'PRT', 'X']


In [13]:
print('there are', len(list(data.training_set.vocab)), 'unique words in the train set') 

there are 50536 unique words in the train set


In [14]:
def pair_counts(sequences_A, sequences_B):
    """Return a dictionary keyed to each unique value in the first sequence list
    that counts the number of occurrences of the corresponding value from the
    second sequences list.
    
    For example, if sequences_A is tags and sequences_B is the corresponding
    words, then if 1244 sequences contain the word "time" tagged as a NOUN, then
    you should return a dictionary such that pair_counts[NOUN][time] == 1244
    """
    
    emission_counts = {}

    for sequence_A, sequence_B in zip(sequences_A, sequences_B):
        
        for a, b in zip(sequence_A, sequence_B):
            
            if a not in emission_counts.keys():    # Sequence A defines the keys of emission_counts 
                emission_counts[a] = {}            # emission_counts is a dictionary with values themselves as dictionary
            
            if b in emission_counts[a].keys():     # Sequence B defines the keys of emission_counts.values (dictionary)
                emission_counts[a][b]+=1
            else:
                emission_counts[a][b]=1
    
    return emission_counts


# Calculate C(t_i, w_i)
emission_counts = pair_counts(list(data.training_set.Y), list(data.training_set.X)) # tags sequence x corresponding words sequence, by sentence in train set

assert len(emission_counts) == 12, \
       "Uh oh. There should be 12 tags in your dictionary."
assert max(emission_counts["NOUN"], key=emission_counts["NOUN"].get) == 'time', \
       "Hmmm...'time' is expected to be the most common NOUN."
HTML('<div class="alert alert-block alert-success">Your emission counts look good!</div>')

In [15]:
max_value = max(emission_counts['NOUN'].values())  # maximum value
max_keys = [k for k, v in emission_counts['NOUN'].items() if v == max_value] # getting all keys containing the `maximum`
print(max_keys, max_value)

['time'] 1275


In [16]:
# Create a lookup table mfc_table where mfc_table[word] contains the tag label most frequently assigned to that word
from collections import namedtuple

FakeState = namedtuple("FakeState", "name")

class MFCTagger:
    # NOTE: You should not need to modify this class or any of its methods
    missing = FakeState(name="<MISSING>")
    
    def __init__(self, table):
        self.table = defaultdict(lambda: MFCTagger.missing)
        self.table.update({word: FakeState(name=tag) for word, tag in table.items()})
        
    def viterbi(self, seq):
        """This method simplifies predictions by matching the Pomegranate viterbi() interface"""
        return 0., list(enumerate(["<start>"] + [self.table[w] for w in seq] + ["<end>"]))


# TODO: calculate the frequency of each tag being assigned to each word (hint: similar, but not
# the same as the emission probabilities) and use it to fill the mfc_table

word_counts = pair_counts(list(data.training_set.X), list(data.training_set.Y))

mfc_table = {}
for word in list(data.training_set.vocab):
    mfc_table[word] = max(word_counts[word], key=word_counts[word].get)

# DO NOT MODIFY BELOW THIS LINE
mfc_model = MFCTagger(mfc_table) # Create a Most Frequent Class tagger instance

assert len(mfc_table) == len(data.training_set.vocab), ""
assert all(k in data.training_set.vocab for k in mfc_table.keys()), ""
assert sum(int(k not in mfc_table) for k in data.testing_set.vocab) == 5521, ""
HTML('<div class="alert alert-block alert-success">Your MFC tagger has all the correct words!</div>')

In [17]:
print('these is the most used POS for a sample of words in the train set')
{k: mfc_table[k] for k in list(mfc_table)[:10]}

these is the most used POS for a sample of words in the train set


{'happen': 'VERB',
 'meeting': 'NOUN',
 'magistrates': 'NOUN',
 '3247': 'NUM',
 'estimate': 'NOUN',
 'Cemetery': 'NOUN',
 'spectrally': 'ADV',
 '2:33': 'NUM',
 'self-conscious': 'ADJ',
 'Into': 'ADP'}

In [18]:
def replace_unknown(sequence):
    """Return a copy of the input sequence where each unknown word is replaced
    by the literal string value 'nan'. Pomegranate will ignore these values
    during computation.
    """
    return [w if w in data.training_set.vocab else 'nan' for w in sequence]

def simplify_decoding(X, model):
    """X should be a 1-D sequence of observations for the model to predict"""
    _, state_path = model.viterbi(replace_unknown(X))
    return [state[1].name for state in state_path[1:-1]]  # do not show the start/end state predictions

In [19]:
for key in data.testing_set.keys[:3]:
    print("Sentence Key: {}\n".format(key))
    print("Predicted labels:\n-----------------")
    print(simplify_decoding(data.sentences[key].words, mfc_model))
    print()
    print("Actual labels:\n--------------")
    print(data.sentences[key].tags)
    print("\n")

Sentence Key: b100-28144

Predicted labels:
-----------------
['CONJ', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'CONJ', 'NOUN', 'NUM', '.', '.', 'NOUN', '.', '.']

Actual labels:
--------------
('CONJ', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'CONJ', 'NOUN', 'NUM', '.', '.', 'NOUN', '.', '.')


Sentence Key: b100-23146

Predicted labels:
-----------------
['PRON', 'VERB', 'DET', 'NOUN', 'ADP', 'ADJ', 'ADJ', 'NOUN', 'VERB', 'VERB', '.', 'ADP', 'VERB', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADP', 'DET', 'NOUN', '.']

Actual labels:
--------------
('PRON', 'VERB', 'DET', 'NOUN', 'ADP', 'ADJ', 'ADJ', 'NOUN', 'VERB', 'VERB', '.', 'ADP', 'VERB', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADP', 'DET', 'NOUN', '.')


Sentence Key: b100-35462

Predicted labels:
-----------------
['DET', 'ADJ', 'NOUN', 'VERB', 'VERB', 'VERB', 'ADP', 'DET', 'ADJ', 'ADJ', 'NOUN', 'ADP', 'DET', 'ADJ', 'NOUN', '.', 'ADP', 'ADJ', 'NOUN', '.', 'CONJ', 'ADP', 'DET', '<MISSING>', 'ADP', 'ADJ', 'ADJ', 

In [20]:
def accuracy(X, Y, model):
    """Calculate the prediction accuracy by using the model to decode each sequence
    in the input X and comparing the prediction with the true labels in Y.
    
    The X should be an array whose first dimension is the number of sentences to test,
    and each element of the array should be an iterable of the words in the sequence.
    The arrays X and Y should have the exact same shape.
    
    X = [("See", "Spot", "run"), ("Run", "Spot", "run", "fast"), ...]
    Y = [(), (), ...]
    """
    correct = total_predictions = 0
    for observations, actual_tags in zip(X, Y):
        
        # The model.viterbi call in simplify_decoding will return None if the HMM
        # raises an error (for example, if a test sentence contains a word that
        # is out of vocabulary for the training set). Any exception counts the
        # full sentence as an error (which makes this a conservative estimate).
        try:
            most_likely_tags = simplify_decoding(observations, model)
            correct += sum(p == t for p, t in zip(most_likely_tags, actual_tags))
        except:
            pass
        total_predictions += len(observations)
    return correct / total_predictions

In [21]:
mfc_training_acc = accuracy(data.training_set.X, data.training_set.Y, mfc_model)
print("training accuracy mfc_model: {:.2f}%".format(100 * mfc_training_acc))

mfc_testing_acc = accuracy(data.testing_set.X, data.testing_set.Y, mfc_model)
print("testing accuracy mfc_model: {:.2f}%".format(100 * mfc_testing_acc))

assert mfc_training_acc >= 0.955, "Uh oh. Your MFC accuracy on the training set doesn't look right."
assert mfc_testing_acc >= 0.925, "Uh oh. Your MFC accuracy on the testing set doesn't look right."
HTML('<div class="alert alert-block alert-success">Your MFC tagger accuracy looks correct!</div>')

training accuracy mfc_model: 95.72%
testing accuracy mfc_model: 93.01%


In [22]:
def unigram_counts(sequences):
    """Return a dictionary keyed to each unique value in the input sequence list that
    counts the number of occurrences of the value in the sequences list. The sequences
    collection should be a 2-dimensional array.
    
    For example, if the tag NOUN appears 275558 times over all the input sequences,
    then you should return a dictionary such that your_unigram_counts[NOUN] == 275558.
    """
    tag_unigrams = {}
    
    for sequence in sequences:
        
        for tag in sequence:
            
            if tag in tag_unigrams.keys():
                tag_unigrams[tag]+=1
            else:
                tag_unigrams[tag]=1
                
    return tag_unigrams

# TODO: call unigram_counts with a list of tag sequences from the training set
tag_unigrams = unigram_counts(list(data.training_set.Y))

assert set(tag_unigrams.keys()) == data.training_set.tagset, \
       "Uh oh. It looks like your tag counts doesn't include all the tags!"
assert min(tag_unigrams, key=tag_unigrams.get) == 'X', \
       "Hmmm...'X' is expected to be the least common class"
assert max(tag_unigrams, key=tag_unigrams.get) == 'NOUN', \
       "Hmmm...'NOUN' is expected to be the most common class"
HTML('<div class="alert alert-block alert-success">Your tag unigrams look good!</div>')

In [23]:
import pprint
pp = pprint.PrettyPrinter(indent=1)
print('frequency of tags in train set:')
pp.pprint(tag_unigrams)

frequency of tags in train set:
{'.': 117757,
 'ADJ': 66754,
 'ADP': 115808,
 'ADV': 44877,
 'CONJ': 30537,
 'DET': 109671,
 'NOUN': 220632,
 'NUM': 11878,
 'PRON': 39383,
 'PRT': 23906,
 'VERB': 146161,
 'X': 1094}


In [24]:
def bigram_counts(sequences):
    """Return a dictionary keyed to each unique PAIR of values in the input sequences
    list that counts the number of occurrences of pair in the sequences list. The input
    should be a 2-dimensional array.
    
    For example, if the pair of tags (NOUN, VERB) appear 61582 times, then you should
    return a dictionary such that your_bigram_counts[(NOUN, VERB)] == 61582
    """
    tag_bigrams = {}

    for sequence in sequences:
        
        l = len(sequence)
        
        for index in range(l-1):
            
            bigram = sequence[index:index+2]
            
            if bigram in tag_bigrams.keys():
                tag_bigrams[bigram] += 1
            else:
                tag_bigrams[bigram] = 1
                
    return tag_bigrams     

# TODO: call bigram_counts with a list of tag sequences from the training set
tag_bigrams = bigram_counts(list(data.training_set.Y))

assert len(tag_bigrams) == 144, \
       "Uh oh. There should be 144 pairs of bigrams (12 tags x 12 tags)"
assert min(tag_bigrams, key=tag_bigrams.get) in [('X', 'NUM'), ('PRON', 'X')], \
       "Hmmm...The least common bigram should be one of ('X', 'NUM') or ('PRON', 'X')."
assert max(tag_bigrams, key=tag_bigrams.get) in [('DET', 'NOUN')], \
       "Hmmm...('DET', 'NOUN') is expected to be the most common bigram."
HTML('<div class="alert alert-block alert-success">Your tag bigrams look good!</div>')

In [25]:
pp = pprint.PrettyPrinter(indent=1)
print('frequency of tags in train set:')
pp.pprint(tag_bigrams)

frequency of tags in train set:
{('.', '.'): 12588,
 ('.', 'ADJ'): 3334,
 ('.', 'ADP'): 7595,
 ('.', 'ADV'): 5124,
 ('.', 'CONJ'): 8174,
 ('.', 'DET'): 8008,
 ('.', 'NOUN'): 9782,
 ('.', 'NUM'): 1412,
 ('.', 'PRON'): 5448,
 ('.', 'PRT'): 2168,
 ('.', 'VERB'): 9041,
 ('.', 'X'): 147,
 ('ADJ', '.'): 6666,
 ('ADJ', 'ADJ'): 3758,
 ('ADJ', 'ADP'): 5895,
 ('ADJ', 'ADV'): 645,
 ('ADJ', 'CONJ'): 2500,
 ('ADJ', 'DET'): 386,
 ('ADJ', 'NOUN'): 43664,
 ('ADJ', 'NUM'): 467,
 ('ADJ', 'PRON'): 249,
 ('ADJ', 'PRT'): 1301,
 ('ADJ', 'VERB'): 1167,
 ('ADJ', 'X'): 31,
 ('ADP', '.'): 1099,
 ('ADP', 'ADJ'): 9533,
 ('ADP', 'ADP'): 2347,
 ('ADP', 'ADV'): 1805,
 ('ADP', 'CONJ'): 217,
 ('ADP', 'DET'): 52841,
 ('ADP', 'NOUN'): 29965,
 ('ADP', 'NUM'): 3467,
 ('ADP', 'PRON'): 8109,
 ('ADP', 'PRT'): 1675,
 ('ADP', 'VERB'): 4690,
 ('ADP', 'X'): 53,
 ('ADV', '.'): 7577,
 ('ADV', 'ADJ'): 6143,
 ('ADV', 'ADP'): 6352,
 ('ADV', 'ADV'): 4336,
 ('ADV', 'CONJ'): 789,
 ('ADV', 'DET'): 3309,
 ('ADV', 'NOUN'): 1478,
 ('ADV', '

In [26]:
def starting_counts(sequences):
    """Return a dictionary keyed to each unique value in the input sequences list
    that counts the number of occurrences where that value is at the beginning of
    a sequence.
    
    For example, if 8093 sequences start with NOUN, then you should return a
    dictionary such that your_starting_counts[NOUN] == 8093
    """
    tag_starts = {}
    
    for sequence in sequences:
        if sequence[0] in tag_starts.keys():
            tag_starts[sequence[0]]+=1
        else:
            tag_starts[sequence[0]]=1

    return tag_starts

# TODO: Calculate the count of each tag starting a sequence
tag_starts = starting_counts(list(data.training_set.Y))

assert len(tag_starts) == 12, "Uh oh. There should be 12 tags in your dictionary."
assert min(tag_starts, key=tag_starts.get) == 'X', "Hmmm...'X' is expected to be the least common starting bigram."
assert max(tag_starts, key=tag_starts.get) == 'DET', "Hmmm...'DET' is expected to be the most common starting bigram."
HTML('<div class="alert alert-block alert-success">Your starting tag counts look good!</div>')

In [27]:
print('frequency of starting tags in train set:')
tag_starts

frequency of starting tags in train set:


{'ADV': 4185,
 'ADP': 5583,
 'ADJ': 1582,
 'PRT': 1718,
 'DET': 9763,
 'PRON': 7318,
 'NOUN': 6469,
 'CONJ': 2282,
 '.': 4107,
 'NUM': 760,
 'VERB': 2080,
 'X': 25}

In [28]:
def ending_counts(sequences):
    """Return a dictionary keyed to each unique value in the input sequences list
    that counts the number of occurrences where that value is at the end of
    a sequence.
    
    For example, if 18 sequences end with DET, then you should return a
    dictionary such that your_ending_counts[DET] == 18
    """
    tag_ends = {}
    
    for sequence in sequences:
        
        if sequence[-1] in tag_ends.keys():
            tag_ends[sequence[-1]]+=1
        else:
            tag_ends[sequence[-1]]=1
    
    return tag_ends

# TODO: Calculate the count of each tag ending a sequence
tag_ends = ending_counts(list(data.training_set.Y))

assert len(tag_ends) == 12, "Uh oh. There should be 12 tags in your dictionary."
assert min(tag_ends, key=tag_ends.get) in ['X', 'CONJ'], "Hmmm...'X' or 'CONJ' should be the least common ending bigram."
assert max(tag_ends, key=tag_ends.get) == '.', "Hmmm...'.' is expected to be the most common ending bigram."
HTML('<div class="alert alert-block alert-success">Your ending tag counts look good!</div>')

In [29]:
print('frequency of ending tags in train set:')
tag_ends

frequency of ending tags in train set:


{'.': 44936,
 'NOUN': 722,
 'NUM': 63,
 'VERB': 75,
 'ADJ': 25,
 'ADV': 16,
 'ADP': 7,
 'DET': 14,
 'CONJ': 2,
 'PRON': 4,
 'PRT': 7,
 'X': 1}

In [30]:
basic_model = HiddenMarkovModel(name="base-hmm-tagger")
import pandas as pd
# TODO: create states with emission probability distributions P(word | tag) and add to the model
# (Hint: you may need to loop & create/add new states)

states = {}

for tag in list(data.tagset):
    
    state_emissions = DiscreteDistribution( {word: occurence / tag_unigrams[tag] for word, 
                                             occurence in emission_counts[tag].items()} )
    
    states[tag] = State(state_emissions, name = tag)
    
    # transition edge from start to each tag
    # For the start tag, how likely is it to be tag_i ? distribution of probabilities for <start> over tags which adds up to 1.
    basic_model.add_transition(basic_model.start, states[tag], tag_starts[tag] / len(data.training_set))
    
    # transition edge from a tag to the end
    # for a tag_i, how likely it is at the end of the sentence or not
    basic_model.add_transition(states[tag], basic_model.end, tag_ends[tag] / tag_ends[tag])
    
basic_model.add_states(*states.values())
    
# TODO: add edges between states for the observed transition frequencies P(tag_i | tag_i-1)
# (Hint: you may need to loop & add transitions

import itertools

for tag_1, tag_2 in itertools.product(data.training_set.tagset, data.training_set.tagset):

    basic_model.add_transition(states[tag_1], states[tag_2], tag_bigrams[(tag_1, tag_2)] / tag_unigrams[tag_1])

# NOTE: YOU SHOULD NOT NEED TO MODIFY ANYTHING BELOW THIS LINE
# finalize the model
basic_model.bake()

assert all(tag in set(s.name for s in basic_model.states) for tag in data.training_set.tagset), \
       "Every state in your network should use the name of the associated tag, which must be one of the training set tags."
assert basic_model.edge_count() == 168, \
       ("Your network should have an edge from the start node to each state, one edge between every " +
        "pair of tags (states), and an edge from each state to the end node.")
HTML('<div class="alert alert-block alert-success">Your HMM network topology looks good!</div>')

In [31]:
hmm_training_acc = accuracy(data.training_set.X, data.training_set.Y, basic_model)
print("training accuracy basic hmm model: {:.2f}%".format(100 * hmm_training_acc))

hmm_testing_acc = accuracy(data.testing_set.X, data.testing_set.Y, basic_model)
print("testing accuracy basic hmm model: {:.2f}%".format(100 * hmm_testing_acc))

assert hmm_training_acc > 0.97, "Uh oh. Your HMM accuracy on the training set doesn't look right."
assert hmm_testing_acc > 0.955, "Uh oh. Your HMM accuracy on the testing set doesn't look right."
HTML('<div class="alert alert-block alert-success">Your HMM tagger accuracy looks correct! Congratulations, you\'ve finished the project.</div>')

training accuracy basic hmm model: 97.54%
testing accuracy basic hmm model: 95.94%


In [32]:
for key in data.testing_set.keys[:3]:
    print("Sentence Key: {}\n".format(key))
    print("Predicted labels:\n-----------------")
    print(simplify_decoding(data.sentences[key].words, basic_model))
    print()
    print("Actual labels:\n--------------")
    print(data.sentences[key].tags)
    print("\n")

Sentence Key: b100-28144

Predicted labels:
-----------------
['CONJ', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'CONJ', 'NOUN', 'NUM', '.', '.', 'NOUN', '.', '.']

Actual labels:
--------------
('CONJ', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'CONJ', 'NOUN', 'NUM', '.', '.', 'NOUN', '.', '.')


Sentence Key: b100-23146

Predicted labels:
-----------------
['PRON', 'VERB', 'DET', 'NOUN', 'ADP', 'ADJ', 'ADJ', 'NOUN', 'VERB', 'VERB', '.', 'ADP', 'VERB', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADP', 'DET', 'NOUN', '.']

Actual labels:
--------------
('PRON', 'VERB', 'DET', 'NOUN', 'ADP', 'ADJ', 'ADJ', 'NOUN', 'VERB', 'VERB', '.', 'ADP', 'VERB', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADP', 'DET', 'NOUN', '.')


Sentence Key: b100-35462

Predicted labels:
-----------------
['DET', 'ADJ', 'NOUN', 'VERB', 'VERB', 'VERB', 'ADP', 'DET', 'ADJ', 'ADJ', 'NOUN', 'ADP', 'DET', 'ADJ', 'NOUN', '.', 'ADP', 'ADJ', 'NOUN', '.', 'CONJ', 'ADP', 'DET', 'NOUN', 'ADP', 'ADJ', 'ADJ', '.', 

In [33]:
column_names = [s.name for s in basic_model.states]
#order_index = [column_names.index(c) for c in column_order]

# re-order the rows/columns to match the specified column order
transitions = basic_model.dense_transition_matrix()

print(transitions)

[[6.60516956e-02 1.74941494e-02 3.98524490e-02 2.68866292e-02
  4.28905751e-02 4.20195407e-02 5.13280653e-02 7.40903989e-03
  2.85867205e-02 1.13759196e-02 4.74398935e-02 7.71337722e-04
  0.00000000e+00 6.17893987e-01]
 [4.99389436e-02 2.81533978e-02 4.41629271e-02 4.83207600e-03
  1.87289767e-02 2.89175401e-03 3.27112816e-01 3.49857285e-03
  1.86540608e-03 9.74655949e-03 8.74268634e-03 2.32239311e-04
  0.00000000e+00 5.00093645e-01]
 [4.74506603e-03 4.11598857e-02 1.01334577e-02 7.79330680e-03
  9.36923865e-04 2.28147438e-01 1.29377528e-01 1.49691937e-02
  3.50115927e-02 7.23201601e-03 2.02496448e-02 2.28833939e-04
  0.00000000e+00 5.00015110e-01]
 [8.44346876e-02 6.84548352e-02 7.07838374e-02 4.83184381e-02
  8.79226192e-03 3.68740110e-02 1.64701687e-02 6.65270008e-03
  2.38026254e-02 1.45423344e-02 1.20740378e-01 4.45742049e-05
  0.00000000e+00 5.00089148e-01]
 [1.00209588e-02 5.52135184e-02 3.63832852e-02 4.51761854e-02
  1.47367042e-04 7.59104007e-02 1.22838616e-01 9.41511656e-03


In [34]:
!!jupyter nbconvert *.ipynb

['Traceback (most recent call last):',
 '  File "C:\\Users\\Amith M\\anaconda3\\Scripts\\jupyter-nbconvert-script.py", line 10, in <module>',
 '    sys.exit(main())',
 '  File "C:\\Users\\Amith M\\anaconda3\\lib\\site-packages\\jupyter_core\\application.py", line 264, in launch_instance',
 '    return super(JupyterApp, cls).launch_instance(argv=argv, **kwargs)',
 '  File "C:\\Users\\Amith M\\anaconda3\\lib\\site-packages\\traitlets\\config\\application.py", line 846, in launch_instance',
 '    app.start()',
 '  File "C:\\Users\\Amith M\\anaconda3\\lib\\site-packages\\nbconvert\\nbconvertapp.py", line 369, in start',
 '    self.convert_notebooks()',
 '  File "C:\\Users\\Amith M\\anaconda3\\lib\\site-packages\\nbconvert\\nbconvertapp.py", line 529, in convert_notebooks',
 '    raise ValueError(',
 "ValueError: Please specify an output format with '--to <format>'.",
 "The following formats are available: ['asciidoc', 'custom', 'html', 'latex', 'markdown', 'notebook', 'pdf', 'python', 'rst

In [35]:
import nltk
from nltk import pos_tag, word_tokenize
from nltk.corpus import brown
import random

nltk.download('brown')
training_corpus = nltk.corpus.brown
training_corpus.tagged_sents()[0]

[nltk_data] Error loading brown: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>


[('The', 'AT'),
 ('Fulton', 'NP-TL'),
 ('County', 'NN-TL'),
 ('Grand', 'JJ-TL'),
 ('Jury', 'NN-TL'),
 ('said', 'VBD'),
 ('Friday', 'NR'),
 ('an', 'AT'),
 ('investigation', 'NN'),
 ('of', 'IN'),
 ("Atlanta's", 'NP$'),
 ('recent', 'JJ'),
 ('primary', 'NN'),
 ('election', 'NN'),
 ('produced', 'VBD'),
 ('``', '``'),
 ('no', 'AT'),
 ('evidence', 'NN'),
 ("''", "''"),
 ('that', 'CS'),
 ('any', 'DTI'),
 ('irregularities', 'NNS'),
 ('took', 'VBD'),
 ('place', 'NN'),
 ('.', '.')]

In [36]:
len(training_corpus.tagged_sents())

57340

In [37]:
corpus = list(nltk.corpus.brown.tagged_sents())

random.seed(42)
random.shuffle(corpus)

split = int(0.8 * len(corpus))

train = corpus[:split]
test = corpus[split:]

print('Train samples:', len(train))
print('Test samples:', len(test))

Train samples: 45872
Test samples: 11468


In [38]:
len(corpus)

57340

In [39]:
corpus[:2]

[[('He', 'PPS'),
  ('let', 'VBD'),
  ('her', 'PPO'),
  ('tell', 'VB'),
  ('him', 'PPO'),
  ('all', 'ABN'),
  ('about', 'IN'),
  ('the', 'AT'),
  ('church', 'NN'),
  ('.', '.')],
 [('China', 'NP'),
  ('never', 'RB'),
  ('tried', 'VBD'),
  ('to', 'TO'),
  ('integrate', 'VB'),
  ('Tibet', 'NP'),
  ('by', 'IN'),
  ('extirpating', 'VBG'),
  ('the', 'AT'),
  ("people's", 'NNS$'),
  ('religion', 'NN'),
  ('and', 'CC'),
  ('institutions', 'NNS'),
  ('.', '.')]]

In [40]:
def rearrange_data(sequences):
    x = []
    y = []
    w = set()
    t = set()
    for sequence in sequences:
        sequence_x = []
        sequence_y = []
        for word, tag in sequence:
            sequence_x.append(word)
            sequence_y.append(tag)
            w.add(word)
            t.add(tag)
        x.append(sequence_x)
        y.append(sequence_y)
    return x, y, w, t

In [41]:
train_x, train_y, train_words, train_tagset = rearrange_data(train)
test_x, test_y, test_words, test_tagset = rearrange_data(test)

In [42]:
print(train_x[0])
print(train_y[0])

['He', 'let', 'her', 'tell', 'him', 'all', 'about', 'the', 'church', '.']
['PPS', 'VBD', 'PPO', 'VB', 'PPO', 'ABN', 'IN', 'AT', 'NN', '.']


In [43]:
print('Train words:', len(train_words), 'number of tags:', len(train_tagset))
print('Unknown test words:', len(test_words.difference(train_words)), 'Unknown test tags', len(test_tagset.difference(train_tagset)))

Train words: 50630 number of tags: 450
Unknown test words: 5427 Unknown test tags 22


In [44]:
def findtags(tag_prefix, tagged_text):
    cfd = nltk.ConditionalFreqDist((tag, word) for (word, tag) in tagged_text if tag.startswith(tag_prefix))
    return dict((tag, cfd[tag].keys()) for tag in cfd.conditions())

In [45]:
tagdict = findtags('NN', nltk.corpus.brown.tagged_words(categories='news'))
for tag in sorted(tagdict):
    print (tag, list(tagdict[tag])[:5])

NN ['investigation', 'primary', 'election', 'evidence', 'place']
NN$ ["ordinary's", "court's", "mayor's", "wife's", "governor's"]
NN$-HL ["Golf's", "Navy's"]
NN$-TL ["Department's", "Commissioner's", "President's", "Party's", "Mayor's"]
NN-HL ['Merger', 'jail', 'Construction', 'fund', 'sp.']
NN-NC ['ova', 'eva', 'aya']
NN-TL ['County', 'Jury', 'City', 'Committee', 'Court']
NN-TL-HL ['Mayor', 'Commissioner', 'City', 'Oak', 'Grove']
NNS ['irregularities', 'presentments', 'thanks', 'reports', 'voters']
NNS$ ["taxpayers'", "children's", "members'", "women's", "years'"]
NNS$-HL ["Dealers'", "Idols'"]
NNS$-TL ["States'", "Women's", "Princes'", "Bombers'", "Falcons'"]
NNS-HL ['Wards', 'deputies', 'bonds', 'aspects', 'Decisions']
NNS-TL ['Police', 'Roads', 'Legislatures', 'Bankers', 'Reps.']
NNS-TL-HL ['Nations']


In [46]:
train_tags = set(train_tagset)
len(train_tags)

450

In [47]:
print(train_tags)

{'HVD', 'PPSS+HVD', 'RP-NC', 'PPS+BEZ-NC', 'FW-IN+NP-TL', 'VB', 'JJR-TL', '---HL', 'VB-TL', 'RBT', 'EX', 'WRB-HL', 'FW-VBD', 'FW-JJ-NC', 'PN$', 'BEM-NC', 'WDT+DO+PPS', 'FW-DTS', 'JJR-NC', 'JJT-HL', 'PPO-NC', 'BE-TL', 'VB+AT', 'FW-NPS', 'VBG-TL', 'FW-CD-TL', 'VB+JJ-NC', 'NP+MD', 'NNS-TL', 'DOZ-HL', 'AT-TL', 'FW-AT+NN-TL', 'PPSS+BEM', "''", 'WQL-TL', 'VB+PPO', 'PP$', 'PPLS', 'PPSS+BER', 'FW-AT', 'CD', 'VB+IN', 'CD$', 'RBR', 'NNS-HL', 'PPSS+HV', 'NN', 'DO-NC', 'VBG', 'NR-NC', 'DO-HL', 'NR$', 'DT+BEZ', 'BER-HL', 'AT-TL-HL', 'NNS$-TL-HL', 'FW-NN$', 'BER-TL', 'VB+RP', 'FW-CC', 'VBN-TL-NC', 'CC-TL', '.', 'VBG-NC', 'HV+TO', '(', 'PPSS-NC', 'RBR-NC', 'TO-NC', 'FW-PP$', 'HVZ*', 'NP-TL-HL', 'EX-NC', 'ABN-TL', 'VBD', 'JJS-TL', 'VBN-TL-HL', 'FW-NN-NC', 'ABL', 'ABN-HL', 'AP$', 'WDT-NC', 'PPO-HL', 'PPL-TL', 'QL-TL', 'DTI', 'CC-TL-HL', 'JJ$-TL', 'FW-WPS', 'WDT+DOD', 'RB-HL', 'DT+MD', 'BER*', 'PP$-NC', 'NNS-TL-NC', 'FW-WDT', ',', 'JJ-HL', 'CC-NC', 'NNS$-HL', 'PPS+HVD', 'FW-QL', 'RB', 'PPL-HL', 'CS-TL',

In [52]:
test_tags = set(test_tagset)
len(test_tags)

316

In [53]:
print(test_tags)

{'HVD', 'PPSS+HVD', 'RBR+CS', 'VB', 'JJR-TL', '---HL', 'VB-TL', 'RBT', 'EX', 'WRB-HL', 'PN$', 'JJT-HL', 'VBG-TL', 'NP+MD', 'NNS-TL', 'DOZ-HL', 'AT-TL', 'FW-AT+NN-TL', 'PPSS+BEM', "''", 'WQL-TL', 'VB+PPO', 'PP$', 'PPLS', 'PPSS+BER', 'FW-AT', 'CD', 'VB+IN', 'CD$', 'RBR', 'NNS-HL', 'PPSS+HV', 'NN', 'VBG', 'NR$', 'DT+BEZ', 'BER-HL', 'FW-NN$', 'BER-TL', 'FW-CC', 'VBN-TL-NC', 'CC-TL', '.', 'VBG-NC', 'HV+TO', '(', 'PPSS-NC', 'TO-NC', 'HVZ*', 'ABN-NC', 'NP-TL-HL', 'ABN-TL', 'VBD', 'JJS-TL', 'VBN-TL-HL', 'FW-NN-NC', 'ABL', 'AP$', 'WDT-NC', 'PPO-HL', 'QL-TL', 'DTI', 'RB-HL', 'DT+MD', 'PP$-NC', 'BER*', 'NNS-TL-NC', 'FW-WDT', 'FW-AT+NP-TL', ',', 'JJ-HL', 'NNS$-HL', 'PPS+HVD', 'RB', 'RP-TL', 'WPS-NC', 'NP-TL', 'FW-VBG', 'JJR', 'PPS', 'BEG', 'OD-HL', 'PN+BEZ', 'CC', 'NN$', 'IN-HL', 'BE-HL', 'PN', 'FW-UH', 'DT-HL', 'ABN', 'DOD*', 'PPS+BEZ', 'NR-TL', 'FW-AT-TL', '``', 'PPSS-HL', 'WP$', "'", 'HVG-HL', 'FW-NP-TL', 'MD+PPSS', 'FW-IN-TL', 'NP$-TL', 'PPSS+BEZ*', 'JJT-NC', 'BEDZ-NC', 'NP$', 'BEZ', 'AT-HL', 

In [54]:
wordtags = nltk.ConditionalFreqDist(nltk.corpus.brown.tagged_words())

In [55]:
print(wordtags["clean"].items())

dict_items([('JJ', 47), ('VB', 15), ('VB-HL', 1), ('RB', 1)])


In [56]:
wtlist = sorted(wordtags.items(), key=lambda x: len(x[1]), reverse=True)
for word, freqs in wtlist[:10]:
    print(word, "\t", len(freqs), list(freqs))

that 	 12 ['CS', 'DT', 'WPS', 'WPO', 'QL', 'DT-NC', 'WPS-NC', 'CS-NC', 'WPS-HL', 'CS-HL', 'NIL', 'WPO-NC']
A 	 11 ['AT', 'AT-TL', 'NN', 'NP', 'AT-HL', 'NP-HL', 'NP-TL', 'AT-TL-HL', 'NN-TL', 'FW-IN', 'AT-NC']
in 	 10 ['IN', 'RP', 'IN-HL', 'IN-TL', 'NIL', 'IN-NC', 'RP-HL', 'RP-NC', 'NN', 'FW-IN']
to 	 10 ['TO', 'IN', 'IN-HL', 'TO-HL', 'TO-NC', 'IN-TL', 'IN-NC', 'NIL', 'NPS', 'QL']
: 	 9 [':', ':-HL', '.', ':-TL', 'IN', '.-HL', 'NP', ',', 'NIL']
a 	 8 ['AT', 'AT-HL', 'AT-NC', 'FW-IN', 'NIL', 'NN', 'FW-IN-TL', 'AT-TL']
for 	 7 ['IN', 'CS', 'IN-TL', 'IN-HL', 'RB', 'NN', 'IN-NC']
it 	 7 ['PPS', 'PPO', 'PPS-HL', 'PPS-NC', 'PPO-HL', 'PPO-NC', 'UH']
well 	 7 ['RB', 'QL', 'UH', 'NN', 'JJ', 'VB', 'QL-HL']
as 	 7 ['CS', 'QL', 'IN', 'CS-HL', 'RB', 'CS-TL', 'NIL']


In [57]:
def bigram_counts_brown(sequences):
    """Return a dictionary keyed to each unique PAIR of values in the input sequences
    list that counts the number of occurrences of pair in the sequences list. The input
    should be a 2-dimensional array.
    
    For example, if the pair of tags (NOUN, VERB) appear 61582 times, then you should
    return a dictionary such that your_bigram_counts[(NOUN, VERB)] == 61582
    """
    tag_bigrams = {}

    for sequence in sequences:
        
        l = len(sequence)
        
        for index in range(l-1):
            
            a,b = sequence[index:index+1], sequence[index+1:index+2]
            
            if (a[0],b[0]) in tag_bigrams.keys():
                tag_bigrams[(a[0],b[0])] += 1
            else:
                tag_bigrams[(a[0],b[0])] = 1
                
    return tag_bigrams     

In [58]:
emission_counts = pair_counts(train_y, train_x)
tag_unigrams = unigram_counts(train_y)
tag_bigrams = bigram_counts_brown(train_y)
tag_starts = starting_counts(train_y)
tag_ends = ending_counts(train_y)

In [59]:
brown_model = HiddenMarkovModel(name="brown-hmm-tagger")

states = dict()
for tag, words in emission_counts.items():
    #n = tag_unigrams[tag]
    #assert n == sum(words.values())
    probs = {w:c / tag_unigrams[tag] for w, c in words.items()}
    emissions = DiscreteDistribution(probs)
    state = State(emissions, name=tag)
    brown_model.add_states(state)
    states[tag] = state

n = sum(tag_starts.values())
for tag, counts in tag_starts.items():
    brown_model.add_transition(brown_model.start, states[tag], counts / n)

for (tag1, tag2), counts in tag_bigrams.items():
    brown_model.add_transition(states[tag1], states[tag2], counts / tag_unigrams[tag1])

for tag, counts in tag_ends.items():
    brown_model.add_transition(states[tag], brown_model.end, counts / tag_unigrams[tag])

brown_model.bake()

print('Edges Brown model:', brown_model.edge_count())

Edges Brown model: 7818


In [60]:
def replace_unknown_brown(sequence, vocabulary):
    """Return a copy of the input sequence where each unknown word is replaced
    by the literal string value 'nan'. Pomegranate will ignore these values
    during computation.
    """
    return [w if w in vocabulary else 'nan' for w in sequence]

def simplify_decoding_brown(X, model, vocabulary):
    """X should be a 1-D sequence of observations for the model to predict"""
    _, state_path = model.viterbi(replace_unknown_brown(X, vocabulary))
    return [state[1].name for state in state_path[1:-1]]  # do not show the start/end state predictions

def accuracy_brown(X, Y, model, vocabulary):
    """Calculate the prediction accuracy by using the model to decode each sequence
    in the input X and comparing the prediction with the true labels in Y.
    
    The X should be an array whose first dimension is the number of sentences to test,
    and each element of the array should be an iterable of the words in the sequence.
    The arrays X and Y should have the exact same shape.
    
    X = [("See", "Spot", "run"), ("Run", "Spot", "run", "fast"), ...]
    Y = [(), (), ...]
    """
    correct = total_predictions = 0
    for observations, actual_tags in zip(X, Y):
        
        # The model.viterbi call in simplify_decoding will return None if the HMM
        # raises an error (for example, if a test sentence contains a word that
        # is out of vocabulary for the training set). Any exception counts the
        # full sentence as an error (which makes this a conservative estimate).
        try:
            most_likely_tags = simplify_decoding_brown(observations, model, vocabulary)
            correct += sum(p == t for p, t in zip(most_likely_tags, actual_tags))
        except:
            pass
        total_predictions += len(observations)
    return correct / total_predictions

In [61]:
training_acc = accuracy_brown(train_x, train_y, brown_model, vocabulary=train_words)
print("training accuracy: {:.2f}%".format(100 * training_acc))

testing_acc = accuracy_brown(test_x, test_y, brown_model, vocabulary=train_words)
print("testing accuracy: {:.2f}%".format(100 * testing_acc))

training accuracy: 97.32%
testing accuracy: 92.52%


In [62]:
tag_bigrams_test = bigram_counts_brown(test_y)

In [64]:
Smooth_Brown_model = HiddenMarkovModel(name="brown-smooth-hmm-tagger")

states = dict()
for tag, words in emission_counts.items():
    n = tag_unigrams[tag]
    assert n == sum(words.values())
    probs = {w:c / n for w, c in words.items()}
    emissions = DiscreteDistribution(probs)
    state = State(emissions, name=tag)
    Smooth_Brown_model.add_states(state)
    states[tag] = state

n = sum(tag_starts.values())
for tag, counts in tag_starts.items():
    Smooth_Brown_model.add_transition(Smooth_Brown_model.start, states[tag], counts / n)

for (tag1, tag2), counts in tag_bigrams.items():
    nominator = counts + 1
    denominator = tag_unigrams[tag1] + len(train_tagset)
    Smooth_Brown_model.add_transition(states[tag1], states[tag2], nominator / denominator)

for tag, counts in tag_ends.items():
    Smooth_Brown_model.add_transition(states[tag], Smooth_Brown_model.end, counts / tag_unigrams[tag])

# NOTE: counts statistics from the test set is not used

new_bigrams = 0
for (tag1, tag2), counts in tag_bigrams_test.items():
    if (tag1, tag2) in tag_bigrams:
        continue
    if tag1 not in states or tag2 not in states:
        continue
    denominator = len(train_tagset)
    if tag1 in tag_unigrams:
        denominator += tag_unigrams[tag1]
    Smooth_Brown_model.add_transition(states[tag1], states[tag2], 1 / denominator)
    new_bigrams += 1

print('Extend transition model including', len(tag_bigrams), 'train bigrams by', new_bigrams, 'new bigrams')
    
Smooth_Brown_model.bake()

print('Edges Brown model with Laplace Smoothing:', Smooth_Brown_model.edge_count())

Extend transition model including 7552 train bigrams by 458 new bigrams
Edges Brown model with Laplace Smoothing: 8276


In [66]:
training_acc = accuracy_brown(train_x, train_y, Smooth_Brown_model, vocabulary=train_words)
print("training accuracy: {:.2f}%".format(100 * training_acc))

testing_acc = accuracy_brown(test_x, test_y, Smooth_Brown_model, vocabulary=train_words)
print("testing accuracy: {:.2f}%".format(100 * testing_acc))

training accuracy: 97.30%
testing accuracy: 93.86%
